<a href="https://colab.research.google.com/github/shaobingbing/AI-For-NLP/blob/master/%E6%96%B0%E9%97%BB%E4%BA%BA%E7%89%A9%E8%A8%80%E8%AE%BA%E6%8F%90%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pyltp

     |████████████████████████████████| 5.3MB 4.4MB/s 
  Created wheel for pyltp: filename=pyltp-0.2.1-cp36-cp36m-linux_x86_64.whl size=32058622 sha256=98cd2383f6eeffaa844bf20d8f7248a3146ac2f6fdce7d224d937ff0a6e9d79b
  Stored in directory: /root/.cache/pip/wheels/fc/3a/35/b11293efb2c77c0e7b6fa574271d51cddd9abd1f634535343c
Successfully built pyltp


### 对句子成分进行解析



In [0]:
import pyltp
from gensim.models import Word2Vec
import jieba
import re
from gensim.models.word2vec import LineSentence
from pyltp import SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
from gensim import models
import numpy as np

model_path = './drive/My Drive/model/ltp_data_v3.4.0/'

cws_model = model_path + "cws.model"
pos_model = model_path + "pos.model"
par_model = model_path + "parser.model"
ner_model = model_path + "ner.model"

def get_word_list(sentence, model):
    # 得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list, model):
    # 得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

### 加载词向量，寻找类似单词

In [11]:
word2vec = Word2Vec.load('./drive/My Drive/model/news_word2vec.model')

def get_similar_word_list(word,top=10): 
    #取得与word最相近的10个词
    word_list = []
    for w in word2vec.wv.most_similar(word,topn=top):
        word_list.append(w[0])
    return word_list

def gather_similar_word(word): 
    #广度优先搜索 取得深度为10 与word 相近的所有词 
    words = [word]
    seen = set()
    depth = 10
    while words and depth > 0:
        word = words.pop(0)
        for w in word:
            if w not in seen:
                seen.add(w)
                words.append(get_similar_word_list(w,10))
            else:
                continue
        depth -= 1
    return list(seen)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
import warnings
warnings.filterwarnings("ignore")

spoken_words = gather_similar_word("说")
spoken_words.append("报道") 
print(spoken_words)

['说法', '特别强调', '坦言', '透露', '告诉', '称', '问', '所说', '了解', '看来', '说', '见到', '相信', '建议', '眼中', '明说', '中说', '来说', '获悉', '介绍', '文说', '地说', '看到', '普遍认为', '指出', '认为', '强调', '表示', '还称', '中称', '直言', '报道']


### get SBV relations in corpus

In [0]:
def find_spoken_word_id_and_sub(spoken_words, word_list, parser_list):
    id_list = []
    
    for sub_id, parse_relation in enumerate(parser_list):
        index, relation = parse_relation
        if relation == 'SBV':
            spoken_word = word_list[index - 1]
            if spoken_word in spoken_words:
                word_id = index - 1
                id_list.append((sub_id, word_id))
    
    # 返回值对应主语和说所在的位置
    return id_list


def get_sentence(news, word_list, idx):
    # 取得说的内容 和 SBV 的宾语成分
    # idx 为表示说的次在新闻中的位置信息
    
    # 说后面第一个开头的汉字对应位置
    index = len("".join(word_list[:idx + 1]))
    
    # 寻找第一个句子
    stop1 = news[index + 1:].find("。")
    stop2 = news[index + 1:].find("!")
    stop3 = news[index + 1:].find("?")
    
    # 检查后面是否还有句子
    stop_list = [stop for stop in [stop1, stop2, stop3] if stop != -1]
    if stop_list == None:
        False
        
    # 返回后面的第一个句子
    stop = min(stop_list)
    
    # 检查后面的句子是否有双引号引起来的句子
    begin = 9999
    end = 9999
    if "“" in news[index + 1:] and "”" in news[index + 1:]:
        begin = news[index + 1:].find("“")
        end = news[index + 1:].find("”")
        
    # 第一种情况,双引号在stop前面,表明说词后面的双引号的句子为要说的内容
    if begin < stop:
        result = news[index + 1 + begin: index + 1 + end + 1]
        
    # 第二种情况，双引号的内容在第一个句子后或者无双引号，说词后的句子即为说的内容
    else:
        sentence = news[index + 2: stop + index + 2].strip()
        result = sentence
        next_id = stop + index + 2
        sim = 1
        # 现在要对该情况进行判断，判断下一个句子和该句之间的关联性，通过检查句子之间的相似度之间的关系
        # 如果相似度大于0.7表明相似(可以自行设定)
        while sim > 0.75 and  next_id < len(news) - 1:
            next_sentence_id = next_id
            if next_sentence_id < len(news) - 1:
                next_sentence, next_id = get_next_sentence(news, next_sentence_id)
                sim = sentence_distance(sentence, next_sentence)
                
            if sim > 0.75:
                result += next_sentence
                sentence = next_sentence
    
    return result


def get_next_sentence(news, index):
    # 取得index后面的第一句话
    stop1 = news[index + 1:].find("。")
    stop2 = news[index + 1:].find("!")
    stop3 = news[index + 1:].find("?")
    
    stop_list = [stop for stop in [stop1,stop2,stop3] if stop != -1]
    if stop_list is None:
        False
    stop = min(stop_list)
    return news[index:index+stop+2],index+stop+2

def cut(sentence):
    return " ".join(jieba.cut(sentence))

def sentence_distance(sentence1, sentence2):
    word_list_1 = cut(sentence1).split()
    word_list_2 = cut(sentence2).split()
    
    vec_1 = 0
    vec_2 = 0
    
    for i in range(len(word_list_1)):
        if word_list_1[i] in word2vec.wv.vocab:
            vec_1 += word2vec.wv[word_list_1[i]]
    
    for i in range(len(word_list_2)):
        if word_list_2[i] in word2vec.wv.vocab:
            vec_2 += word2vec.wv[word_list_2[i]]
            
    return np.dot(vec_1, vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))

def get_sub_and_view(idxs, news, word_list):
    # 取得主语以及要说的内容
    sub = []
    speech = []
    for sub_id, spoken_id in idxs:
        sub.append(word_list[sub_id])
        speech.append(get_sentence(news, word_list, spoken_id))
        
    return sub,speech

### 测试可行性

In [0]:
text = "英国央行行长卡尼周二指出，目前不是加息良机，加息将是受限且渐进的，未来几个月将关注消费增长疲弱被其他需求因素抵消的程度，以及经济如何对脱欧谈判做出反应。美联储副主席Fischer(永久投票权)称，美国金融系统比危机之前要强，不良贷款减少。长期低利率可能已经抬高了住房价格。需要进一步努力加强住房金融系统。宏观审慎工具可以缓解住房市场危机。不准备对货币政策和美国经济前景置评。美国芝加哥联储主席Evans(有投票权)称，可能将下一次加息推迟到12月份，以留出时间评估数据。看到多个月份的通胀数据改善至关重要。根据近期数据，通胀难以达到2%。希望见到通胀上行，而不只是平稳。"


In [76]:
word_list = get_word_list(text,cws_model)
postag_list = get_postag_list(word_list,pos_model)
parser_list = get_parser_list(word_list,postag_list,par_model)

idx=find_spoken_word_id_and_sub(spoken_words,word_list,parser_list)
idx

sub,speech = get_sub_and_view(idx,text,word_list)

for i in range(len(sub)):
    print(sub[i],speech[i])

卡尼周二 前不是加息良机，加息将是受限且渐进的，未来几个月将关注消费增长疲弱被其他需求因素抵消的程度，以及经济如何对脱欧谈判做出反应。
Fischer 国金融系统比危机之前要强，不良贷款减少。


In [55]:
for i in range(len(sub)):
    print(sub[i],speech[i])

卡尼周二 前不是加息良机，加息将是受限且渐进的，未来几个月将关注消费增长疲弱被其他需求因素抵消的程度，以及经济如何对脱欧谈判做出反应。
Fischer 国金融系统比危机之前要强，不良贷款减少。
